In [10]:
import numpy as np
import sacc
import matplotlib.pyplot as plt
import pyccl as ccl
import scipy
import importlib as imp
import astropy.io.fits as fits

import CM_code.spurious_george as sp
imp.reload(sp)
import CM_code.halo_model as halo
imp.reload(halo)
import CM_code.lsst_coZmology as zed
imp.reload(zed)
import CM_code.tjp_helper as tjp_helper
imp.reload(tjp_helper)

# import main.py from TJPCov folder
import tjpcov as tjp
from tjpcov.covariance_calculator import CovarianceCalculator


data_dir = '/home/b7009348/WGL_project/DES-data/'
lens_file = data_dir+'DES_Y1A1_3x2pt_redMaGiC_zerr_CATALOG.fits'
source_Zs = data_dir+'mof_BPZ_(stacking).fits'
im3_file_full = data_dir+'y1_im3_shapes_matched.fits'

plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['cmr10']
plt.rcParams['mathtext.fontset'] ='cm'
plt.rcParams['figure.facecolor'] = 'white'
plt.rc('axes', unicode_minus=False)
plt.rc('axes.formatter', use_mathtext=True)

### I'm going to start in the case with n IA for the sake of comparison to SRD forecasting
- Initially I will just look at single $\gamma_t$ covaraince for specific redshift bins

In [13]:
# create LSST sacc file
s = sacc.Sacc()

th_lsst = np.logspace(np.log10(2.5),np.log10(250),10) #forecasts

# start by adding redshift data (this will be used internally by TJP
# to create WL and NC tracers
z_l, dndz_l, zleff = zed.get_dndz_spec('lens')
s.add_tracer('NZ','lens', z=z_l, nz=dndz_l)

z_s, dndz_s, zseff = zed.get_dndz_spec('source')
s.add_tracer('NZ','source', z=z_s, nz=dndz_s)

lensTracer = ccl.NumberCountsTracer(zed.cosmo_SRD, has_rsd=False, dndz=(z_l, dndz_l), 
                                    bias=(z_l, sp.lens_bias(z_l))
shearTracer = ccl.WeakLensingTracer(zed.cosmo_SRD, (z_s, dndz_s), has_shear=True)

ell = np.unique(np.geomspace(180/(th_lsst[-1]/60.) - 10, 180/(th_lsst[0]/60.) + 10, 1024).astype(int))

Cl = ccl.angular_cl(zed.cosmo_SRD, lensTracer, shearTracer, ell)

LSST_xi = ccl.correlation(zed.cosmo_SRD, ell, Cl, th_lsst/60., type='NG')

s.add_theta_xi('galaxy_shearDensity_xi_t', 'lens', 'source', th_lsst/60., LSST_xi)

s.save_fits('forecast_MEM_data_lsst-y1.fits', overwrite=True)

In [ ]:
lsst_config = tjp_helper.generate_config(cosmo=zed.cosmo_SRD, sacc_file='forecast_MEM_data_lsst-y1.fits',
                                   cov_type='RealGaussianFsky', Nlen=18., Nsrc=10., sig_e=0.13, len_bias=1.5, IA=0.,
                                   add_keys=[['ProjectedReal',['lmax', 42427]],['GaussianFsky',['fsky', 0.436]]])

lsstX_config = tjp_helper.generate_config(cosmo=zed.cosmo_SRD, sacc_file='forecast_MEM_data_lsst-y1.fits',
                                   cov_type='RealGaussianFsky', Nlen=18., Nsrc=10., sig_e=np.sqrt(0.2*0.13*0.13), len_bias=1.5, IA=0.,
                                   add_keys=[['ProjectedReal',['lmax', 42427]],['GaussianFsky',['fsky', 0.436]]])

cov = CovarianceCalculator(lsst_config)
cov_same = cov.get_covariance()

cov = CovarianceCalculator(lsstX_config)
cov_cross = cov.get_covariance()

test_cov = 2.*cov_same - 2.*cov_cross

Computing independent covariance blocks
('lens', 'source') ('lens', 'source')


/home/b7009348/WGL_project/LSST_forecast_code/tjpcov/covariance_builder.py:897: UserWarning: Missing n_ell_coupled info for some tracers in the sacc file. You will have to pass it withthe cache
  warnings.warn(


Building the covariance: placing blocks in their place
('lens', 'source') ('lens', 'source')
Computing independent covariance blocks
('lens', 'source') ('lens', 'source')
Building the covariance: placing blocks in their place
('lens', 'source') ('lens', 'source')


In [ ]:
np.linalg.cholesky(test_cov)

LinAlgError: Matrix is not positive definite